# the libraries:

In [1]:
import numpy as np
import cv2 as cv

# Function to Creating an array of square vertices in the Pythagorean tree recursively

In [2]:
# Function to Creating an array of square vertices in a Pythagorean tree
def draw_pythagorean_tree(x, y, side, fi, alfa, deep, count_deep, squares):
    """
    Parameters:
    - x, y: Coordinates of the starting point for drawing the current square.
    - side: Length of the current square.
    - fi: Initial angle of the current branch up is pi/2(90 degrees).
    - alfa: Angle between the branches(pi/2 is isosceles right triangle).
    - deep: Depth or recursion level, indicating how many more levels of branches to draw.
    - count_deep: Counter tracking the current recursion depth.
    - squares: A list to store the vertices of squares representing branches.
    """
    # Calculate the displacement along the x and y axes based on the current angle and side length
    dx = side * np.sin(fi)
    dy = side * np.cos(fi)

    # Calculate the coordinates of the four vertices of the square representing the current branch
    x1, y1 = x + dx, y - dy
    x2, y2 = x + dx - dy, y - dy - dx
    x3, y3 = x - dy, y - dx
    x4, y4 = x - dy + side * np.cos(alfa) * np.sin(fi - alfa), y - dx - side * np.cos(alfa) * np.cos(fi - alfa)

    # Store the vertices of the square in the 'squares' list
    squares.append(np.array([[x, y], [x1, y1], [x2, y2], [x3, y3]], np.int32).reshape((-1, 1, 2)))

    # If the recursion depth is greater than 1, call the function recursively for the two lower branches
    if deep > 1:
        draw_pythagorean_tree(x4, y4, side * np.sin(alfa), fi - alfa + np.pi / 2, alfa, deep - 1, count_deep + 1, squares)
        draw_pythagorean_tree(x3, y3, side * np.cos(alfa), fi - alfa, alfa, deep - 1, count_deep + 1, squares)


# We will create the arrays of squares using the function we built

In [3]:
# Initialize parameters for Pythagorean tree
x, y = 600, 550
side, deep = 70, 10
squares = []

# Generate Pythagorean tree
draw_pythagorean_tree(x , y , side, np.pi/2 , np.pi/5 , deep, 1, squares)

# Drawing using OpenCV

## 1.Determined  the size of the picture, the horizon line and the size of the moon, as well as the size of the big stars

In [4]:
#Fixed
width   = 900
height  = 600
Skyline = 551
bigStarR= 7
MoonSize = 65
MoonGlow = 85
moonCen = (200, 150)

## 2.The painting:

In [5]:
# Create a black image (3 for rgb bat in OpenCV bgr)
img = np.zeros((height, width, 3), dtype=np.uint8)

# Draw background of the night
cv.rectangle(img, (0, 0), (width, Skyline), (0, 0, 0), -1)

# Draw color gradient for the green gras
for i in range(Skyline, height):
    color_ratio = (height - i) / 49
    color = (
        int(125 + color_ratio * (150 - 125)),
        int(179 + color_ratio * (255 - 179)),
        int(60 + color_ratio * (70 - 60))
    )
    img[i, :] = color

# Draw random stars
for i in range(7):
    star_x = np.random.randint(0, width, size=50)
    star_y = np.random.randint(0, Skyline, size=50)
    img[star_y, star_x] = (255-i, 255-2*i, 255-1.5*i)

# Draw the Moon glow
for r in range(MoonGlow, MoonGlow-30, -1):  # Radius for the halo
    color_ratio = (MoonGlow - r) / 30  # Interpolate from white to black
    color = (
        int(color_ratio * 255),  
        int(color_ratio * 255),  
        int(color_ratio * 255)   
    )
    cv.circle(img, moonCen, r, color, -1)
# Draw the Moon
cv.circle(img, moonCen, MoonSize, (255, 255, 255), -1)
cv.circle(img, moonCen, MoonSize-10, (245, 255, 255), -1)

# Draw Tree with gradual color change
for i, sq in enumerate(squares):
    color_ratio = i / len(squares)
    color = (int(255 * (1 - color_ratio)), int(255 * color_ratio), 0)
    cv.fillPoly(img, [sq], color)

# big stars
for center, num in [((760, 79), bigStarR), ((845, 240), bigStarR), ((15, 13), bigStarR)]:
    for r in range(11, 4, -1):
        color_ratio = (11 - r) / bigStarR
        color = (int(color_ratio * 255), int(color_ratio * 255), int(color_ratio * 255))
        cv.circle(img, center, num, color, -1)
        num -= 1


## 3.Text

In [6]:
#text
font=cv.FONT_HERSHEY_TRIPLEX
cv.putText(img, "A lonely Pythagorean tree named ", (120,50), font, 1.3, (245, 255, 245), 2)
cv.putText(img, " Nisan is looking for a job", (275,100), font, 1.3, (245, 255, 245), 2)
cv.putText(img, " Python drawn with OpenCV.", (20,580), font, 0.7, (255,255,255), 1);

# We will conveniently close the image window by pressing any button and we will also display the image

In [7]:
# Show the image and close at the press of any button
cv.imshow("A lonely Pythagorean tree named Nissan is looking for a job", img)
cv.waitKey(0)
cv.destroyAllWindows()

# To save img in the project file uncomment next line:


In [8]:
#cv.imwrite("A lonely Pythagorean tree named Nissan is looking for a job.png", img)

True